In [ ]:
from NeuralNet import NeuralNet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit
from sklearn.datasets import fetch_openml

In [ ]:
NN = NeuralNet(depth=3, learning_rate=0.1, s_in=784, s_out=10)

In [ ]:
X, y = fetch_openml("mnist_784", return_X_y=True)

In [ ]:
y = np.array([[1 if i == int(x) else 0 for i in range(10)] for x in y])

In [ ]:
loss = NN.train(X, y, 1)

In [ ]:
x = np.arange(len(loss))
plt.plot(x, loss)

In [ ]:
NN.forward(X[5])

In [ ]:
y[5]